# Exercise: Titanic Database
    
The JSON file `titanic.json` contains information about 1000+ passengers that were aboard the Titanic when it sunk in 1912. It's a document database: not all entries have the same fields, since there is a lot of missing data.

In [18]:
import json
from pymongo import MongoClient

**1) Load the JSON file into a MongoDB database.**

In [19]:
client = MongoClient('localhost', 27017)
db = client.test_database
db.collection.drop()  # This clears the collection
print('db is a', type(db))
print('db.collection is a', type(db.collection))

db is a <class 'pymongo.database.Database'>
db.collection is a <class 'pymongo.collection.Collection'>


In [20]:
with open('titanic.json', 'r') as data_file:  # Remember to change the file path as needed
    data_json = json.load(data_file)
    db.collection.insert_many(data_json)

**2) Find out exactly how many passengers (number of documents) there are in the database.**

In [21]:
len(list(db.collection.find(
    {}
)))

1309

**3) How old was passenger "Bourke, Mr. John"? (use the fields `name` and `age`)**

In [23]:
list(db.collection.find({"name": 'Bourke, Mr. John'},{"age": 1,"_id": 0}))


[{'age': 40.0}]

**4) The field `survived` tells us whether a passenger survived (value 1) or not (value 0). Find out how many survived and how many did not (note: many have missing data).**

In [24]:
len(list(db.collection.find(
    {'survived': 1}
)))

342

In [25]:
len(list(db.collection.find(
    {'survived': 0}
)))

549

**5) Who was the oldest survivor of the Titanic?**

In [34]:
list(db.collection.aggregate([
    {'$match': {'survived': 1}},
    {'$group': {'_id': 1, 'max_survivor_age': {'$max': '$age'}}}
]))

[{'_id': 1, 'max_survivor_age': 80.0}]

In [37]:
list(db.collection.find(
    {'age': 80.0}
))

[{'_id': 631,
  'survived': 1.0,
  'name': 'Barkworth, Mr. Algernon Henry Wilson',
  'gender': 'male',
  'age': 80.0,
  'n_siblings_spouse': 0.0,
  'n_parents_children': 0.0,
  'ticket_number': '27042',
  'fare': 30.0,
  'cabin': 'A23',
  'hometown': 'Hessle, East Yorkshire, England, UK',
  'boarded_from': 'Southampton',
  'destination': 'New York, New York, US',
  'lifeboat': 'B',
  'class': 1.0}]

**6) Find the survival rate (survivors/total) for each ticket class. (use the field `class`. There were three: 1, 2, 3)**

In [51]:
surv_class1=len(list(db.collection.find({"survived": 1, "class":1})))/len(list(db.collection.find({"class":1})))
surv_class2= len(list(db.collection.find({"survived": 1, "class":2})))/len(list(db.collection.find({"class":2})))
surv_class3=len(list(db.collection.find({"survived": 1, "class":3})))/len(list(db.collection.find({"class":3})))
#adds up to 341, there is 1 person that is not assigned a class.
print(f"The survival rate for class 1 is {surv_class1}")
print(f"The survival rate for class 2 is {surv_class2}")
print(f"The survival rate for class 3 is {surv_class3}")


The survival rate for class 1 is 0.42024539877300615
The survival rate for class 2 is 0.3161764705882353
The survival rate for class 3 is 0.1671388101983003


**7) Which five passengers paid the five highest ticket prices? (use the field `fare`)**

In [56]:
list_top_fares= list(db.collection.find().sort("fare", -1).limit(5))
for passenger in list_top_fares:
        print(f"name: {passenger['name']}, fare: {passenger['fare']}")


name: Lesurer, Mr. Gustave J, fare: 512.3292
name: Ward, Miss. Anna, fare: 512.3292
name: Cardeza, Mrs. James Warburton Martinez (Charlotte Wardle Drake), fare: 512.3292
name: Cardeza, Mr. Thomas Drake Martinez, fare: 512.3292
name: Fortune, Miss. Mabel Helen, fare: 263.0
